<a href="https://colab.research.google.com/gist/JDubWeuu/6eb267e8abff7129720ed98dadd14ccd/1d_cnn_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from keras.metrics import Accuracy
from sklearn.metrics import accuracy_score
import pandas as pd

In [ ]:
emg_data = pd.read_csv('data.csv')
emg_data.head()
#drop time
emg_data = emg_data.drop('time', axis=1)
#drop any of the samples that have 0 as a channel value
channels_to_check = ['channel1', 'channel2', 'channel3', 'channel4', 'channel5', 'channel6', 'channel7', 'channel8']  # Add or remove channels as needed

# Filter out rows where any of the specified channels have a zero value
emg_data = emg_data[~(emg_data[channels_to_check] == 0).any(axis=1)]
X = emg_data.drop('class', axis=1)  # Assuming 'class' is your target variable
y = emg_data['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
zero_values_check = (emg_data[channels_to_check] == 0).sum()
zero_values_check

channel1    0
channel2    0
channel3    0
channel4    0
channel5    0
channel6    0
channel7    0
channel8    0
dtype: int64

In [ ]:
#Normalize the data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [ ]:
#Conduct balancing of dataset classes (to prevent oversampling if there's more of a occurence of certain classes compared to others)
smote = SMOTE(sampling_strategy='auto')
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)
X_train_balanced_df = pd.DataFrame(X_train_balanced, columns=X_train_scaled_df.columns)
y_train_balanced_df = pd.Series(y_train_balanced)
X_train_balanced_df.head()

,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8
0,0.535088,0.512500,0.577143,0.458874,0.470588,0.558011,0.664634,0.556122
1,0.495614,0.466667,0.582857,0.480519,0.486275,0.513812,0.524390,0.505102
2,0.570175,0.520833,0.622857,0.493506,0.494118,0.535912,0.475610,0.489796
3,0.504386,0.483333,0.668571,0.506494,0.490196,0.502762,0.518293,0.505102
4,0.557018,0.425000,0.571429,0.432900,0.439216,0.497238,0.542683,0.602041


In [ ]:
model = Sequential()
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(X_train_balanced.shape[1], 1)))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=y_train_balanced.nunique(), activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
history = model.fit(X_train_balanced_df, to_categorical(y_train_balanced), epochs=100, batch_size=32)

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_25 (Conv1D)          (None, 6, 16)             64        
                                                                 
 conv1d_26 (Conv1D)          (None, 4, 32)             1568      
                                                                 
 max_pooling1d_23 (MaxPooli  (None, 2, 32)             0         
 ng1D)                                                           
                                                                 
 flatten_12 (Flatten)        (None, 64)                0         
                                                                 
 dense_18 (Dense)            (None, 128)               8320      
                                                                 
 dense_19 (Dense)            (None, 7)                 903       
                                                     

KeyboardInterrupt: ignored

In [ ]:
# prompt: test my model above with my test dataset for x and y and save model

test_loss, test_acc = model.evaluate(X_test_scaled_df, to_categorical(y_test), verbose=2)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)
model.save('my_model.h5')


174/174 - 0s - loss: 0.3251 - accuracy: 0.9128 - 287ms/epoch - 2ms/step
Test loss: 0.3250885009765625
Test accuracy: 0.9128398895263672


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')